**HELPER FILE TO MAKE SOME TUNING AND ADJUSTMENTS ON THE THYMIO**

In [9]:
from tdmclient import ClientAsync, aw
import time

import numpy as np
import math

from obstacle_avoidance import *
from PIL import Image
from utils.data import *
from map import *
from A_star_alg import *
from motion_control import *


In [ ]:
client = ClientAsync()
node = await client.wait_for_node()
aw(node.lock())


In [ ]:
#import map from image and set start/goal
im = Image.open('b.png', 'r')
im = im.convert('L')
grid = np.array(im)
grid = (grid !=255)*1
max_x, max_y = grid.shape[0], grid.shape[1] # Size of the map

goal = Point(1,7)
start = Point(19,7)
base_dir = 0
size_thym = 5

thym = Robot(start, base_dir)
obs = list()
for i in range(max_x):
    for j in range(max_y):
        if grid[i][j]:
            obs.append(Point(i,j))

corners = [Point(0,0), Point(max_x,0), Point(0, max_y), Point(max_x, max_y)]

map = Map(corners, obs)
env = Environment(thym, map, goal)

path = calculate_path(env, size_thym, False)

print(len(path))
direction = base_dir
for i in range(len(path)-1):
    direction = move_step(path[i+1].x-path[i].x, path[i+1].y-path[i].y, direction)

In [181]:
def test_control(pos, goal, orientation):
    GAIN = 2
    SPEED = 200
    METER = 14700

    dist = ((goal[0]-pos[0])**2+(goal[1]-pos[1])**2)**(1/2)
    if (goal[0]-pos[0]) == 0:
        if goal[1]-pos[1] > 0:
            angle = 180
        else:
            angle = 0
    else:
        angle = 180*((goal[1]-pos[1]) > 0) + math.atan((goal[1]-pos[1])/(goal[0]-pos[0]))/np.pi*180 - orientation
    if angle > 180:
        angle -= 360
    speedR = int(SPEED - GAIN * angle)
    speedL = int(SPEED + GAIN * angle)
    v = {
    "motor.left.target": [speedL],
    "motor.right.target": [speedR],
    }
    aw(node.set_variables(v))
    aw(client.sleep(METER//(10/dist)/1000)) 
    v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
    }
    aw(node.set_variables(v))
        
pos = (0,0)
goal = (0,1)
orient = 0



test_control(pos, goal, orient)


1.0


In [153]:
v = {
"motor.left.target": [0],
"motor.right.target": [0],
}
aw(node.set_variables(v))


In [37]:
goal = (0,0)
pos = (2,1)
orientation = 0

dist = ((goal[0]-pos[0])**2+(goal[1]-pos[1])**2)**(1/2)
print(dist)
if (goal[0]-pos[0]) == 0:
    if goal[1]-pos[1] > 0:
        angle = 180
    else:
        angle = 0
else:
    angle = 180*((goal[1]-pos[1]) > 0) + math.atan((goal[1]-pos[1])/(goal[0]-pos[0]))/np.pi*180 - orientation
if angle > 180:
    angle -= 360


2.23606797749979
26.565051177077986


In [2]:
await node.unlock()

{'error_code': 2}